Medium - Line Reflection (Easy with hint)

Given n points on a 2D plane, find if there is such a line parallel to the y-axis that reflects the given points symmetrically.

In [ ]:
class Solution:
    def isReflected(self, points: List[List[int]]) -> bool:
        if not points:
            return True
    
        points_set = {tuple(p) for p in points}
        
        xs = [x for x, _ in points]
        midLine = (min(xs) + max(xs)) / 2 
        
        for x, y in points_set:
            otherX = 2 * midLine - x
            if (otherX, y) not in points_set:
                return False
        return True

Medium - Length of the longest alphabetical continuous substring

Give hint that in python ord(a) gives numerical ordering

In [ ]:
class Solution:
    def longestContinuousSubstring(self, s: str) -> int:
        #sliding window approach
        left = 0
        ans = 0
        for rightIndex, val in enumerate(s):
            if ord(val) - ord(s[rightIndex-1 if rightIndex>0 else 0]) != 1:
                left = rightIndex
            ans = max(ans, rightIndex-left+1)
        return ans

Easy (Greedy) - Count Days Spent Together

Alice and Bob are traveling to Rome for separate business meetings.
You are given 4 strings arriveAlice, leaveAlice, arriveBob, and leaveBob. Alice will be in the city from the dates arriveAlice to leaveAlice (inclusive), while Bob will be in the city from the dates arriveBob to leaveBob (inclusive). Each will be a 5-character string in the format "MM-DD", corresponding to the month and day of the date.
Return the total number of days that Alice and Bob are in Rome together.
You can assume that all dates occur in the same calendar year, which is not a leap year. Note that the number of days per month can be represented as: [31, 28, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31].

In [ ]:
class Solution:
    def countDaysTogether(self, arriveAlice: str, leaveAlice: str, arriveBob: str, leaveBob: str) -> int:
        D = [31, 28, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31]
        
        def helper(days):
            return int(days[-2:]) + sum(D[:int(days[:2]) - 1])
        
        return max(0, helper(min(leaveAlice, leaveBob)) - helper(max(arriveAlice, arriveBob)) + 1)

Medium - Word Pattern 2 (I think its a hard since it requires backtracking algo)

Given a pattern and a string s, return true if s matches the pattern.
A string s matches a pattern if there is some bijective mapping of single characters to non-empty strings such that if each character in pattern is replaced by the string it maps to, then the resulting string is s. A bijective mapping means that no two characters map to the same string, and no character maps to two different strings.

In [ ]:
from collections import defaultdict
class Solution:
    def wordPatternMatch(self, pattern: str, s: str) -> bool:
        mapping = defaultdict()
        used = set()
        def helper(x,y):
            if x == len(pattern):
                return y == len(s)
            symbol = pattern[x]
            if symbol in mapping:
                word = mapping[symbol]
                if s[y : y + len(word)] != word:
                    return False
                return helper(x + 1, y + len(word))
            for k in range(y + 1, len(s) + 1):
                new_word = s[y:k]
                if new_word in used:
                    continue
                mapping[symbol] = new_word
                used.add(new_word)
                if helper(x + 1, y + len(new_word)):
                    return True
                del mapping[symbol]
                used.remove(new_word)
            return False
        return helper(0, 0)

Medium: Range Sum Query 2D (Binary Indexed Tree)

Given a 2D matrix matrix, handle multiple queries of the following types:
Update the value of a cell in matrix.
Calculate the sum of the elements of matrix inside the rectangle defined by its upper left corner (row1, col1) and lower right corner (row2, col2).
Implement the NumMatrix class:
NumMatrix(int[][] matrix) Initializes the object with the integer matrix matrix.
void update(int row, int col, int val) Updates the value of matrix[row][col] to be val.
int sumRegion(int row1, int col1, int row2, int col2) Returns the sum of the elements of matrix inside the rectangle defined by its upper left corner (row1, col1) and lower right corner (row2, col2).

Can make it a lot harder by asking if it is possible to optimize
- Use bit manipulation to construct indexed tree

In [ ]:
class NumMatrix:

    def __init__(self, matrix: List[List[int]]):
        self.matrix = matrix

    def update(self, row: int, col: int, val: int) -> None:
        self.matrix[row][col] = val

    def sumRegion(self, row1: int, col1: int, row2: int, col2: int) -> int:
        ans = 0
        for y in range(row1, row2+1):
            for x in range(col1, col2+1):
                ans += self.matrix[y][x]
        return ans

In [ ]:
from typing import List

class NumMatrix:
    def __init__(self, matrix: List[List[int]]):
        self.rows = len(matrix)
        if self.rows == 0:
            self.cols = 0
            self.bit = []
            self._empty = True
            return
        self._empty = False
        self.cols = len(matrix[0])
        # 1-based indexing: allocate (rows+1) x (cols+1)
        self.bit = [[0] * (self.cols + 1) for _ in range(self.rows + 1)]
        self._buildBIT(matrix)

    @staticmethod
    def _lsb(n: int) -> int:
        # rightmost set bit
        return n & -n

    def _updateBIT(self, r: int, c: int, val: int) -> None:
        i = r
        while i <= self.rows:
            j = c
            while j <= self.cols:
                self.bit[i][j] += val
                j += self._lsb(j)
            i += self._lsb(i)

    def _queryBIT(self, r: int, c: int) -> int:
        s = 0
        i = r
        while i > 0:
            j = c
            while j > 0:
                s += self.bit[i][j]
                j -= self._lsb(j)
            i -= self._lsb(i)
        return s

    def _buildBIT(self, matrix: List[List[int]]) -> None:
        for i in range(1, self.rows + 1):
            for j in range(1, self.cols + 1):
                self._updateBIT(i, j, matrix[i - 1][j - 1])

    def update(self, row: int, col: int, val: int) -> None:
        if self._empty:
            return
        # current value at (row, col) via single-cell sumRegion
        old_val = self.sumRegion(row, col, row, col)
        diff = val - old_val
        # convert to 1-based and push diff
        self._updateBIT(row + 1, col + 1, diff)

    def sumRegion(self, row1: int, col1: int, row2: int, col2: int) -> int:
        if self._empty:
            return 0
        # convert to 1-based
        r1, c1, r2, c2 = row1 + 1, col1 + 1, row2 + 1, col2 + 1
        # inclusion-exclusion (matches your Java layout):
        a = self._queryBIT(r2, c2)
        b = self._queryBIT(r1 - 1, c1 - 1)
        c = self._queryBIT(r2, c1 - 1)
        d = self._queryBIT(r1 - 1, c2)
        return (a + b) - (c + d)


Medium: Nested List Weight Sum

You are given a nested list of integers nestedList. Each element is either an integer or a list whose elements may also be integers or other lists.
The depth of an integer is the number of lists that it is inside of. For example, the nested list [1,[2,2],[[3],2],1] has each integer's value set to its depth.
Return the sum of each integer in nestedList multiplied by its depth.

Input: nestedList = [[1,1],2,[1,1]]
Output: 10
Explanation: Four 1's at depth 2, one 2 at depth 1. 1*2 + 1*2 + 2*1 + 1*2 + 1*2 = 10.

To make it simplier, we have 3 premade functions we can call, nestedList.isInteger(), nestedList.getInteger(), nestedList.getList()

In [ ]:
def depthSum(self, nestedList: List[NestedInteger]) -> int:
    def helper(curr_list, curr_depth):
        ans = 0
        for item in curr_list:
            if item.isInteger():
                ans += item.getInteger()*curr_depth
            else:
                ans += helper(item.getList(), curr_depth+1)
        return ans
    return helper(nestedList, 1)

Medium - Number of Boomerangs (Hashmap) - I think this is a medium-hard question

You are given n points in the plane that are all distinct, where points[i] = [xi, yi]. A boomerang is a tuple of points (i, j, k) such that the distance between i and j equals the distance between i and k (the order of the tuple matters).
Return the number of boomerangs.

In [ ]:
from collections import defaultdict
class Solution:
    def numberOfBoomerangs(self, points: List[List[int]]) -> int:
        ans = 0
        for point in points:
            mapping = defaultdict()
            for q in points:
                x = point[0] - q[0]
                y = point[1] - q[1]
                mapping[x*x + y*y] = 1 + mapping.get(x*x + y*y, 0)
            for k in mapping:
                ans += mapping[k] * (mapping[k]-1)
        return ans


Medium K-diff pairs in an array

Given an array of integers nums and an integer k, return the number of unique k-diff pairs in the array.
A k-diff pair is an integer pair (nums[i], nums[j]), where the following are true:
0 <= i, j < nums.length
i != j
|nums[i] - nums[j]| == k
Notice that |val| denotes the absolute value of val.

Example 1:
Input: nums = [3,1,4,1,5], k = 2
Output: 2
Explanation: There are two 2-diff pairs in the array, (1, 3) and (3, 5).
Although we have two 1s in the input, we should only return the number of unique pairs.

brute force approach is too slow. edge case if k = 0, need 2 of an identical keys to work

In [ ]:
#2 pointer approach (copied from editorial)
class Solution:
    def findPairs(self, nums: List[int], k: int) -> int:
        nums = sorted(nums)
        left = 0
        right = 1
        result = 0
        while (left < len(nums) and right < len(nums)):
            if (left == right or nums[right] - nums[left] < k):
                # List item 1 in the text
                right += 1
            elif nums[right] - nums[left] > k:
                # List item 2 in the text
                left += 1
            else:
                # List item 3 in the text
                left += 1
                result += 1
                while (left < len(nums) and nums[left] == nums[left - 1]):
                    left += 1
        return result

In [ ]:
#using counter
from collections import Counter
class Solution:
    def findPairs(self, nums: List[int], k: int) -> int:
        counter = Counter(nums)
        ans = 0
        for item, count in counter.items():
            if k > 0 and item + k in counter.keys():
                ans += 1
            elif k == 0 and count > 1:
                ans += 1
        return ans

Easy - Largest Subarray Length K

An array A is larger than some array B if for the first index i where A[i] != B[i], A[i] > B[i].

For example, consider 0-indexing:

[1,3,2,4] > [1,2,2,4], since at index 1, 3 > 2.
[1,4,4,4] < [2,1,1,1], since at index 0, 1 < 2.
A subarray is a contiguous subsequence of the array.

Given an integer array nums of distinct integers, return the largest subarray of nums of length k.

In [ ]:
#Greedy solution, since each number is unique we can use it
class Solution:
    def largestSubarray(self, nums: List[int], k: int) -> List[int]:
        n = len(nums)
        ans_index = 0
        for i in range(n-k+1):
            if nums[i] > nums[ans_index]:
                ans_index = i
        return nums[ans_index: ans_index+k]


Medium (greedy) - Find the value of the partition

You are given a positive integer array nums.
Partition nums into two arrays, nums1 and nums2, such that:

Each element of the array nums belongs to either the array nums1 or the array nums2.
Both arrays are non-empty.
The value of the partition is minimized.
The value of the partition is |max(nums1) - min(nums2)|.

Here, max(nums1) denotes the maximum element of the array nums1, and min(nums2) denotes the minimum element of the array nums2.
Return the integer denoting the value of such partition.

In [ ]:
class Solution:
    def findValueOfPartition(self, nums: List[int]) -> int:
        nums.sort()
        n = len(nums)
        ans = float('inf')
        for i in range(n-1):
            ans = min(ans, nums[i+1] - nums[i])
        return ans